In [101]:
import numpy as np
from random import randint
from math import floor, fabs
from cplex.callbacks import UserCutCallback
import cplex  as CPX
import cplex.callbacks as CPX_CB
import sys

def SecModelParameter ( VarsValue, m, n, my_rhs, my_coef, BranchedVariables,MaxMultiply):
#     print 'BranchedVariables',BranchedVariables
    NotBranched=list()
    for i in range (n):
        if i not in (BranchedVariables):
            NotBranched.append(i)
    MultiplyVar=list()
    c=[]
    d=(np.zeros(((2 * m) + (4 * n), 2*n)))
    halatest=[]
    a=[]
    newrow=[]
    zer=(np.zeros(((2 * m) + (4 * n), 2*n)))

    f=0
    while len(MultiplyVar)<MaxMultiply:
        if f not in BranchedVariables:
            MultiplyVar.append(f)
        f=f+1
    MultiplyVar.sort()
    print 'MultiplyVar', MultiplyVar
    NodeNum=n-len(MultiplyVar)
    
    print 'NodeNum', NodeNum
    for i in MultiplyVar:   
        for h in range (m):
            c.append(0)
        for h in range (m):
            lin=0            
            for j in range (n):
#                 print 'i', i, 'h', h,'j',j
#                 print 'j1',j
                if j in (BranchedVariables):
#                     print 'j2',j
                    sv = VarsValue[BranchedVariables.index(j)]
#                     print 'sv', sv
                    lin =lin+ (sv * my_coef[h][j]) 
#                     print 'lin', lin
#                 print 'sv'
            lin=lin- my_rhs[h]
            c.append(lin)                
        for h in range (n):
            c.append(0)
        for h in range (n):
            c.append(0)

        for jj in range (n):
            lin2=-1
            if jj in (BranchedVariables):
                sv = VarsValue[BranchedVariables.index(jj)]
                lin2 =lin2+ sv
            c.append(lin2)
            
        for jj in range (n):
            lin2=0
            if jj in (BranchedVariables):
                lin2 =-1*VarsValue[BranchedVariables.index(jj)]
            c.append(lin2)

              
    k=NodeNum-1 

    for i in MultiplyVar: 
        k=k+1
        for t in range(m):              
            for j in range(n):    
                d[(k-NodeNum)*(2*m+4*n)+t][j+n]=my_coef[t][j] 
                
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t][n+i]=d[(k-NodeNum)*(2*m+4*n)+t][n+i]-my_rhs[t]   
                
        for t in range(m):  
            for kk in NotBranched:
                d[(k-NodeNum)*(2*m+4*n)+t+m][kk]=my_coef[t][kk] 
                

        for t in range(m):  
            for j in range(n):
                d[(k-NodeNum)*(2*m+4*n)+t+m][j+n]=-1*my_coef[t][j] 
                 
        for t in range(m):  
            d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]=d[(k-NodeNum)*(2*m+4*n)+t+m][n+i]+my_rhs[t] 
                
        for t in range(n):              
            d[(k-NodeNum)*(2*m+4*n)+t+2*m][i]=-1 
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+t][n+t]=1 
        
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+n+t][n+t]=-1 
        
        for mm in NotBranched: 
             
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+mm][mm]=1  
            
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]=d[(k-NodeNum)*(2*m+4*n)+t+2*n+2*m][i]+1            
            

        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+2*n+t][n+t]=-1 

        for t in NotBranched:  
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][t]=-1  
               
        
        for t in range(n):  
            
            d[(k-NodeNum)*(2*m+4*n)+2*m+3*n+t][n+t]=1 
            
        d=np.vstack([d, zer])                          
            
    d = np.delete(d, slice(((n - NodeNum)*((2 * m) + (4 * n))),((n-NodeNum+1)*((2 * m) + (4 * n)))), axis=0)
#     print 'c',c
#     for j in range (50):
#         print 'j',j
#         for i in range (749):    
#         print 'd',j,d[:,j]
    print len(d[:,1])
    print len(d[1,:])
#     print 'unbranched',unbranched
    print c,111111111111111111111111, d
    return c, d,MultiplyVar,NotBranched
#     print c, d,unbranched

In [102]:
def SecondModel (cc,dd, BranchedVariables,MultiplyVar,NotBranched):
#     NotBranched=list()
#     for i in range (n):
#         if i not in (BranchedVariables):
#             NotBranched.append(i)
#     print 'cc,dd,NotBranched',cc,dd,NotBranched
    numrows = len(dd)    # 3 rows in your example
    numcols = len(dd[0])
    VarNum=numcols/2
    LenMultiplyVar=len(MultiplyVar)
    lenNotbranched=len(NotBranched)
    ConstNum=numrows/LenMultiplyVar
    print 'ConstNum',ConstNum
    my_obj=[]
    my_ub=[]
    my_lb=[]
    my_colnames=[]
    my_rownames=["c1"]
    my_coef=[] 
    my_row=[]
    my_rhs=[]
    my_sense=""
    my_Nmah=list()
    
    for i in range(numrows):
        my_obj=cc
        my_ub.append(1)
        my_lb.append(0)
        my_colnames.append("y"+str(i))
    sumarray=[]     
    NotBranched.sort()    
#     sumshode=list()
    for t in range (VarNum):
        sumlist=list()
        for j in range(numrows):
#             print 't,j',t,j
            if t in MultiplyVar: 
                indj=MultiplyVar.index(t)
#                 print 'indj',indj
                indjrange=range(indj*ConstNum,(indj+1)*ConstNum)
#                 print 'indjrange',indjrange
                if j in indjrange:
#                     print 'dd[j,t]+dd[j,t+VarNum]',j,'ll',dd[j,t]+dd[j,t+VarNum]
                    sumlist.append(dd[j,t]+dd[j,t+VarNum])
                    dd[j,t+VarNum]=0
                else:
                    sumlist.append(dd[j,t])
            else:
                sumlist.append(dd[j,t])
        sumarray.append(sumlist)
    print 'sumarray',sumarray
#     print 'newDD',dd
  
#     print 'notbranched',NotBranched
   
    
    for t in range(LenMultiplyVar):  
        for i in range (VarNum):
            sumlist=list()
            if t>0:
                for p in range(t):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                        
            for j in range(ConstNum):
                sumlist.append(dd[t*ConstNum+j,i+VarNum])
            if i in MultiplyVar and i > MultiplyVar[t]:
                indx=MultiplyVar.index(i)
                indy=MultiplyVar.index(MultiplyVar[t])
                
                for x in range(indx-indy-1):
                    for j in range (ConstNum):
                        sumlist.append(0) 
                for j in range(ConstNum):
                    sumlist.append(dd[(indx)*ConstNum+j,MultiplyVar[t]+VarNum])
                    dd[(indx)*ConstNum+j,MultiplyVar[t]+VarNum]=0
#                 sumarray.append([t+indx,indy])
                if t<LenMultiplyVar-2 :
                    for p in range(LenMultiplyVar-indx-1):
                        for j in range (ConstNum):
                            sumlist.append(0)
            else:
                if t<LenMultiplyVar-1 :
#                     print 'lenNotbranched',lenNotbranched
                    for p in range(LenMultiplyVar-t-1):
                            for j in range (ConstNum):
                                sumlist.append(0) 
#             print 't,i',t,',',i
#             print 'sumlist p2',sumlist
#             print 'lensumlist p2',len(sumlist)
            sumarray.append(sumlist)
    print 'part2' ,sumarray  
    
    rows=[]
    for j in range(numrows):
        my_row.append(1)
    rows.append([range(numrows),my_row])  
    def populatebyrow(SecProb):
        SecProb.objective.set_sense(SecProb.objective.sense.maximize)
        SecProb.variables.add(obj = my_obj,lb=my_lb, names = my_colnames)
        
        for i in range (len(sumarray)):
            if all(k == 0 for k in sumarray[i])!=True:
#                 print 'i',i
                sumline=[]
                sumline.append([range(len(sumarray[i])),sumarray[i]])
                print 'sumline',sumline
                SecProb.linear_constraints.add(lin_expr = sumline, senses =  "E",
                                rhs = [0], names = ["u"+str(i)])
        SecProb.linear_constraints.add(lin_expr = rows, senses = "L",
                                             rhs = [1], names = ["t0"])
   
    my_SecProb = CPX.Cplex()
    handle = populatebyrow(my_SecProb)
    my_SecProb.solve()
    my_SecProb.write("lpex2.lp")
    SecVariables    = my_SecProb.solution.get_values()
    SecObjective=my_SecProb.solution.get_objective_value()
#     print 'SecVariables,SecObjective,my_colnames',SecVariables, SecObjective,my_colnames
    return SecVariables, SecObjective,my_colnames

In [103]:
def GeneratintCutParameters ( m, n, my_rhs, my_coef, BranchedVariables,SecVariables,MaxMultiply,MultiplyVar): 
    BranchedLen=len(BranchedVariables)
#     unbranched=list()
#     for i in range (n):
#         if i not in (BranchedVariables):
#             unbranched.append(i)
#     f=0
#     while len(unbranched)<MaxMultiply:
#         if f not in BranchedVariables:
#             unbranched.append(f)
#         f=f+1
#     unbranched.sort()
    print 'unbranched2222', MultiplyVar
    LenMultiplyVar=len(MultiplyVar)                   
#     NodeNum=n-len(unbranched)
#     k=NodeNum-1 
    lhs=(np.zeros((LenMultiplyVar*((2 * m) + (4 * n)),BranchedLen)))
    rhs=(np.zeros((LenMultiplyVar*((2 * m) + (4 * n)),1)))

    for i in range (LenMultiplyVar):   
        for h in range (m):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+m+h][BranchedVariables.index(j)]=my_coef[h][j]*SecVariables[i*(2*m+4*n)+m+h]  
                rhs[i*(2*m+4*n)+m+h]=my_rhs[h]*SecVariables[i*(2*m+4*n)+m+h]  
            
        for h in range (n):
            for j in (BranchedVariables):
                lhs[i*(2*m+4*n)+2*m+2*n+j][BranchedVariables.index(j)]=SecVariables[i*(2*m+4*n)+2*m+2*n+j]  
                rhs[i*(2*m+4*n)+2*m+2*n+h]=SecVariables[i*(2*m+4*n)+2*m+2*n+h]  
            
#         for h in range (n):
        for j in (BranchedVariables):
            lhs[i*(2*m+4*n)+2*m+3*n+j][BranchedVariables.index(j)]=-1*SecVariables[i*(2*m+4*n)+2*m+3*n+j]  

    #     print 'lefttt',lhs,'rightttt', rhs
    LenLeft=len(lhs)
    LenRight=len(rhs)
    LenSec=len(SecVariables)
#     print 'LenLeft',LenLeft
#     print'LenRight',LenRight
#     print 'LenSec',LenSec
#     indicesLhs=[]
#     for j in range (2):
#         indicesLhs.append([i for i in range(LenLeft) if lhs[i][j]!=0]) 
# #     print indicesLhs
#     indicesRhs = [(i,rhs[i]) for i in range(LenRight) if rhs[i]>0]
#     print 'indicesRhs',indicesRhs
#     indiceSecVariables=[(i , SecVariables[i]) for i in range(LenSec) if SecVariables[i]!=0]
#     ValueSecVariables=[SecVariables[i] for i in range(LenSec) if SecVariables[i]!=0]
#     print 'positiveLeftHand',indicesLhs 
#     print 'indiceSecVariables',indiceSecVariables
#     print 'ValueSecVariables', ValueSecVariables
    FinalLhs=np.sum(lhs, axis=0)
    FinalRhs=np.sum(rhs, axis=0)
    print 'FinalLhs',FinalLhs, 'FinalRhs',FinalRhs
    return FinalLhs,FinalRhs



In [104]:
  
c = CPX.Cplex('NewExample.lp')
m=c.linear_constraints.get_num()
n=c.variables.get_num()
my_rhs=[]
my_coef=[]
for i in range(m):
    my_row=[]
    for j in range(n):
        my_row.append(c.linear_constraints.get_coefficients(i,j))
    my_coef.append(my_row)

for i in range (m):
    my_rhs.append(c.linear_constraints.get_rhs(i))
values=[ 1.0]
branched=[3]
MaxMultiply=2
SecModelParameterOutput=SecModelParameter(values, m, n, my_rhs, my_coef,branched,MaxMultiply)
SecondModelOutput=SecondModel (SecModelParameterOutput[0],SecModelParameterOutput[1],branched,SecModelParameterOutput[2],SecModelParameterOutput[3])
print SecondModelOutput[0] ,SecondModelOutput[1]  ,SecondModelOutput[2]                     

MultiplyVar [0, 1]
NodeNum 2
48
8
[0, 0, 0, 0, -28.0, -151.0, -27.0, -167.0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0.0, 0, 0, 0, -1.0, 0, 0, 0, 0, -28.0, -151.0, -27.0, -167.0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, 0.0, 0, 0, 0, -1.0] 111111111111111111111111 [[   0.    0.    0.    0.  -52.  -76.   26.   72.]
 [   0.    0.    0.    0.    3.   68.   43.  -61.]
 [   0.    0.    0.    0. -107.  -46.   98.   53.]
 [   0.    0.    0.    0.  -78.  -76.   49.  -97.]
 [  48.  -76.   26.    0.   52.   76.  -26.  -72.]
 [  93.   68.   43.    0.   -3.  -68.  -43.   61.]
 [ -27.  -46.   98.    0.  107.   46.  -98.  -53.]
 [  -8.  -76.   49.    0.   78.   76.  -49.   97.]
 [  -1.    0.    0.    0.    1.    0.    0.    0.]
 [  -1.    0.    0.    0.    0.    1.    0.    0.]
 [  -1.    0.    0.    0.    0.    0.    1.    0.]
 [  -1.    0.    0.    0.    0.    0.    0.    1.]
 [   0.    0.    0.    0.   -1.    0.    0.    0.]
 [   0.    0.    0.    0.    0.   -1.    0.    0.]
 [   0.    0.    0.    0.    0

Tried aggregator 1 time.
LP Presolve eliminated 0 rows and 8 columns.
Reduced LP has 9 rows, 40 columns, and 160 nonzeros.
Presolve time = 0.00 sec. (0.03 ticks)
Initializing dual steep norms . . .
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 0.0 ['y0', 'y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y7', 'y8', 'y9', 'y10', 'y11', 'y12', 'y13', 'y14', 'y15', 'y16', 'y17', 'y18', 'y19', 'y20', 'y21', 'y22', 'y23', 'y24', 'y25', 'y26', 'y27', 'y28', 'y29', 'y30', 'y31', 'y32', 'y33', 'y34', 'y35', 'y36', 'y37', 'y38', 'y39', 'y40', 'y41', 'y42', 'y43', 'y44', 'y45', 'y46', 'y47']
